In [1]:
# Import dependencies
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import unidecode 
from config import api_key
import requests
import json

In [2]:
breeds_url = 'https://stacker.com/stories/2454/origins-50-most-popular-dog-breeds'

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/ellengrove/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


In [3]:
browser.visit(breeds_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
results = soup.find_all('div', class_='ct-slideshow__slide__text-container')
results.pop(0)

<div class="ct-slideshow__slide__text-container">
<h2 class="ct-slideshow__slide__text-container__caption">
<div>Origins of the 50 most popular dog breeds</div>
</h2>
<div class="ct-slideshow__slide__text-container__description">
<div><p>Dogs have long been known as man's best friend. Science tells us why: European scientists studying the DNA of prehistoric dogs in Germany concluded in 2017 that all modern dogs <a href="https://www.sciencedaily.com/releases/2017/07/170718113516.htm" rel="noopener" target="_blank">have the same geographic origin</a> and can be traced back to a single instance of domestication about 20,000 to 40,000 years ago. This makes them the first animal domesticated by man. The mutually beneficial relationship between human and dog became inseparable from there, as dogs provided safety and security to their nomadic hunter-gatherer owners and humans returned the favor with food and shelter. And in each other, both parties found loyal companionship.<br/><br/>
Modern 

In [5]:
len(results)

50

In [6]:
# initialize empty list to store breeds
breed_list = []
rank_list = []
origin_list = []
latitude_list = []
longitude_list = []


for item in results:

    breed_title = item.find(class_="ct-slideshow__slide__text-container__caption").text
    breed_rank = breed_title.split('. ',1)
    
    rank_list.append(breed_rank[0].strip())
    breed_list.append(unidecode.unidecode(breed_rank[1].strip()))
    para = item.find('div',class_="ct-slideshow__slide__text-container__description").\
        get_text(separator='<br>', strip=True).split('<br>')
    origin = para[0].split(': ',1)
    origin_list.append(origin[1])

In [33]:
latitude_list = []
longitude_list = []
for i in range(len(origin_list)):
    location = origin_list[i]
    try: 
        base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
        url = f'{base_url}address={location}&key={api_key}'
        data = requests.get(url).json()
        
        lat = data['results'][0]['geometry']['location']['lat']
        lon = data['results'][0]['geometry']['location']['lng']
        latitude_list.append(lat)
        longitude_list.append(lon)
        
    except:
        latitude_list.append(np.NaN)
        longitude_list.append(np.NaN)

In [34]:
origin_df = pd.DataFrame(list(zip(breed_list, rank_list,origin_list,
                                 latitude_list,longitude_list)),
               columns =['breed_name', 'rank','origin','lat','lng'])

In [35]:
origin_df

,breed_name,rank,origin,lat,lng
0,English cocker spaniel,#50,England,52.355518,-1.174320
1,Portuguese water dog,#49,Portugal,39.399872,-8.224454
2,St. Bernard,#48,Switzerland,46.818188,8.227512
3,Akita,#47,Japan,36.204824,138.252924
4,Chesapeake Bay retriever,#46,"Maryland, USA",39.045755,-76.641271
5,Shiba Inu,#45,Japan,36.204824,138.252924
6,West Highland white Terrier,#44,Scotland,56.490671,-4.202646
7,Bichon Frise,#43,Spain,40.463667,-3.749220
8,Rhodesian ridgeback,#42,Africa,-8.783195,34.508523
9,Belgian Malinois,#41,Belgium,50.503887,4.469936


In [8]:
# with open('data/top_dogs.json', 'w') as outfile:
#     json.dump(dog_data, outfile,indent=4)